## Metropolis Hastings Algorithm ##
#### As follows the the reader finds a description of the implementation of Metropolis Hastings Algoritm. Some pieces of code have been commented out, they are meant to swap between real and generated datasets, and beween the informative and noninformative priors as described in the report results. Currently, the code is set to return evaluations on generated data by using informative priors. Also, after the main algorithmic function, auxiliary methods are defined in the order they are called. ####

### Preliminary Steps

In [ ]:
### Import Relevant Libraries ###

import numpy as np
from scipy import stats
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.discrete.discrete_model import Probit

In [ ]:
    #INITIALIZE REAL DATA
    
    #Retreive the Data from the excel file in excelpath, for simplicity we import only the first 50 lines
    # notes, X must be full rank, the excel file has been handed in with the report
#     df = pd.read_excel(excelpath)
#     Y = df.loc[:49,'Is Male?'].values.astype(np.float64)
#     X1 = pd.DataFrame(np.ones(50), columns = ['intercept'])
#     X2 = df.loc[:49, ['education_level', 'hourly_income', 'restaurant_per_day', 'coffees_per_day']]
#     X = pd.merge(X1, X2, left_index = True, right_index = True).values.astype(np.float64)

In [ ]:
#GENERATE TEST DATA

#Generate X as uniform (0,1) distributed, let the true beta be [0.14, 0.12, 0.06, 0.38, 0.3], use it generate 
#the bernulli probabilities and sample the vector Y. Again, include a column of ones in X
X1 = np.random.uniform(0, 1, (50,4))
X2 = np.ones(50)
X = np.insert(X1, 0, X2, axis = 1)
real_beta = np.array([0.14, 0.12, 0.06, 0.38, 0.3])
p = sp.stats.norm.cdf(X.dot(real_beta))
Y = np.zeros(len(p))
for i in range(len(p)):
    Y[i] = np.random.binomial(1, p[i])

### Metropolis Hastings Algorithm

##### Define Functions for the MH-Algorithm

In [ ]:
def q(X, Y, tau, Beta, precision):

    #Action: return a random Beta_candidate from N(Beta, tau*{[I(Beta)]^-1})

    return np.random.multivariate_normal(mean = Beta, cov = variance(X, Y, tau, Beta, precision)) 

def variance(X, Y, tau, Beta, precision):
    
    #Action : returns the inverse of the Fisher Score Matrix evaluated at that Beta
    
    # Generate the Fisher Score Matrix, stabilize p when it grows too close to 1 or 0, same goes for the second term
    # to ensure convergence of the inversion algorthm
    p = sp.stats.norm.cdf(X.dot(Beta))
    p[p>1-precision] = 1-precision
    p[p<0+precision] = 0+precision
    
    W = (1/(p*(1-p)))*np.diag(sp.stats.norm.pdf(sp.stats.norm.ppf(p))**2)
    I = X.T @ W @ X
    
    # Transform the Fisher Score Matrix (I) into the variance of the proposal distribution
    var = (tau * np.linalg.inv(I))
    return var

def alpha(X, Y, Beta, Beta_candidate):
    
    #Action: given the current value of Beta and its candidate value, 
    #        computs the alpha according to the formula from the slides 
    
    return min(1, posterior(X, Y, Beta_candidate)/posterior(X, Y, Beta)) 

def posterior(X, Y, Beta):

    # Action: returns the posterior probability of Beta as computed through the formula on the slides
    
    return (((sp.stats.norm.cdf(X.dot(Beta)))**Y)*((1-sp.stats.norm.cdf(X.dot(Beta)))**(1-Y))).prod() * prior(Beta) 
    
def prior(Beta):
    # Action: returns the prior probability of Beta as specified by the Decision Maker, 
    # in this case we pick a multinariate normal with high variance
    
      # NON_INFORMATIVE PRIORS: the following code is to be used when the prior is meant to be noninformative
#     Cov = np.array([[ 0., -9.90046474, 5.52025854, -0.3783414,  4.72113058],
#                     [ 9.18048934,  0., 4.44209061,  6.03586419, 7.85665877],
#                     [ 5.91231272, -5.92272759, 0., -7.59668468, 5.67554438],
#                     [ 1.10847662, -6.87587458, -9.43656676, 0., 1.54521851],
#                     [ 9.15512171, -1.85204579, -3.93767116, 3.04618583, 0.]], dtype = np.float64)
#     return sp.stats.multivariate_normal.pdf(Beta, np.ones(len(Beta)), 2000 * np.identity(len(Beta)) +  Cov) 
    
    # INFORMATIVE PRIORS: the following code is to be used when the prior is meant to be informative    
    return sp.stats.multivariate_normal.pdf(Beta, np.ones(len(Beta)), 2 * np.identity(len(Beta)))  

def diagnostics(result, num_accepted, num_output):
    
    #Diagnostic 1: Acceptance rate
    rate_accepted = num_accepted / (num_output + 1)
    print(f'the acceptance rate is: {rate_accepted}')
    
    # Diagnostic 2: Time Series of the Marginal Distributions of Beta Entries, visual inspection for convergence
    figure, axis = plt.subplots(len(result[0]))
    Beta_series = [[] for p in range(len(result[0]))]
    time_index = [t for t in range(len(result))]
    
    for p in range(len(result[0])):
        for t in range(len(result)):
            Beta_series[p].append(result[t][p])
        axis[p].plot(time_index, Beta_series[p])
        #axis[p].set_title(f'Beta {p}')
        
    figure.set_figheight(15)
    figure.set_figwidth(15)
    plt.subplots_adjust(left=1.1, bottom=1.1, right=1.9, top=1.9, wspace=0.4, hspace=0.4)

    #Diagnostic 3: Means and variances of the Betas
    means = [0 for _ in range(len(result[0]))]
    variances = [0 for _ in range(len(result[0]))]
    for j in range(len(result[0])):
        for x in result:
            means[j] += x[j]
        means[j] = means[j]/len(result)
    for j in range(len(result[0])):
        for x in result:
            variances[j] += (x[j] - means[j])**2
        variances[j] = variances[j]/(len(result)-1)
    print(f'the means of betas are: {means}')
    print(f'the variances of betas are: {variances}')

##### Define the MH-Algorithm 

In [ ]:
def MH_Algo(tau, excelpath, num_output, precision):

    # initiate iteration to 0,
    # initialize the number of Betas accepted to 0
    # set a random seed for reproducibility, 
    # check whether precision is small enough
    np.random.seed(1)
    Beta = Probit(Y, X).fit().params
    result = []
    iteration = 0
    num_accepted = 0
    if precision > 1e-1:
        raise ValueError (f'precision must be lower than 0.1 while it now is {precision}')
    
    while iteration <= num_output:
        iteration += 1
        Beta_candidate = q(X, Y, tau, Beta, precision)
        u = np.random.rand()
        if u < alpha(X, Y, Beta, Beta_candidate):
            Beta = Beta_candidate
            num_accepted += 1
            if iteration > 0.2*num_output: #do not include the warm up period in the estimation of posterior functionals
                result.append(list(Beta))
    
    #DIAGNOSTICS:
    
    diagnostics(result, num_accepted, num_output)

#### In the following tab, the user can parametrize and call the algorithm to view its results, obviously, the excel file path has to be tailored to the one on the user's  computer ####

In [ ]:
tau = 1e-3
num_output = 10000
excelpath = ".\Statistics and Probability\\DataBase for MH-Algorithm.xlsx"
precision = 1e-15
MH_Algo(tau, excelpath, num_output, precision)

#### In this last tab, the user finds the code used to conjecture the relationship beteween tau and the acceptance rate. ####

In [ ]:
### RELATION TAU, ACCEPTANCE RATE ### 

# Note. To run this script the MH-Alorithm must be changes so that no diagnostics are run
# and the function actually return the acceptance rate. 

num_output = 100
precision = 1e-15
excelpath = ".\Statistics and Probability\\DataBase for MH-Algorithm.xlsx" 
TU = np.arange(-10, 10, 0.0001)
AR = np.zeros(len(TU))
i = 0
for tau in TU:
    try:
        AR[i] = MH_Algo(tau, excelpath, num_output, precision)
    except Exception:
        AR[i] = AR[i-1]
    i += 1
    
plt.plot(TU, AR)
plt.title('Acceptance Rate As a Function of Tau')
plt.xlabel('tau')
plt.ylabel('acceptance rate')